In [ ]:
#To remove file in Databricks

In [ ]:
%fs rm /mnt/itv-github-db/sandbox/students -r

In [ ]:
students1_str = """
{"students": [{"student_id":1,"student_first_name":"Eduino","student_last_name":"Dawdry","student_email":"edawdry0@whitehouse.gov","student_gender":"Bigender","student_phone_numbers":["5737119029"],"student_address":{"street":"218 Ridgeway Crossing","city":"Omaha","state":"Nebraska","postal_code":"68110"}},
{"student_id":2,"student_first_name":"Lacee","student_last_name":"Prosek","student_email":"lprosek1@barnesandnoble.com","student_gender":"Polygender","student_phone_numbers":["9526294997","4699651256","7167123799","7061046839","7013761528"],"student_address":{"street":"188 Meadow Vale Avenue","city":"Augusta","state":"Georgia","postal_code":"30919"}},
{"student_id":3,"student_first_name":"Richart","student_last_name":"Zimmer","student_email":"rzimmer2@ox.ac.uk","student_gender":"Non-binary","student_phone_numbers":["3129072019","2815879465","9793774370","6367833815"],"student_address":{"street":"87155 Lunder Court","city":"Fort Myers","state":"Florida","postal_code":"33994"}},
{"student_id":4,"student_first_name":"Elyse","student_last_name":"Addionisio","student_email":"","student_gender":"Polygender","student_phone_numbers":["7347984926","3364474838","7136381150"],"student_address":{"street":"77 Sugar Alley","city":"Atlanta","state":"Georgia","postal_code":"31132"}},
{"student_id":5,"student_first_name":"Lilian","student_last_name":"Warret","student_email":"","student_gender":"Male","student_phone_numbers":["5031246553","6151432197","2152754201"],"student_address":{"street":"82540 Summer Ridge Point","city":"Sioux Falls","state":"South Dakota","postal_code":"57193"}}
]}
"""

In [ ]:
import json
students1 = json.loads(students1_str)

In [ ]:
from pyspark.sql import Row
students1_df = spark.createDataFrame(Row(**x) for x in students1['students'])

Write in delta format 
# usecase of delta files: internally delta format is parquet format but it alsio exoposes the logs

In [ ]:
student_df.write.format('delta').save('location')

read file from delta

In [ ]:
student_df=spark.read.format('delta').load('/mnt/sdatabricks/students')

In [ ]:
from delta.tables import DeltaTable
# load delta files
student_delta = DeltaTable.forPath(spark,'/mnt/sdatabricks/students')
student_delta
#<delta.tables.DeltaTable at 0x7f7fa3a151c0>

In [ ]:
# To show the delta file loaction
%fs ls /mnt/sdatabricks/students

# To show the delta file logs
%fs ls /mnt/sdatabricks/students/_delta_log/

In [ ]:
#convert delta variable to Dataframe
student_delta.toDF().printSchema()

# update the existing the value in delta lake

In [ ]:
from pyspark.sql.functions import lit, expr
student_delta.update(
    condition = expr("student_id == 4 "),
  set = {"student_email": lit("edision@berkley.com")})

# Delta logs will create the logs folder having lof files for the activities for over the time period where JSON and CRC file tells the 
# update and merge file and over the time we will be having lots of files so we need vaccuming of those files

In [ ]:
# Delete the data in delta format

In [ ]:
help(student_delta.delete)

In [1]:

student_delta.delete(

    condition = expr("student_id==5")
)

NameError: name 'student_delta' is not defined

In [ ]:
#sql style syntax:
student_delta.delete(
    condition = "student_id = 5",
)

In [ ]:
## Upsert : Update and Merge

In [ ]:
students2_str = """
{"students": [
{"student_id":4,"student_first_name":"Elyse","student_last_name":"Addionisio","student_email":"eaddionisio3@berkeley.edu","student_gender":"Polygender","student_phone_numbers":["7347984926","3364474838","7136381150"],"student_address":{"street":"77 Sugar Alley","city":"Atlanta","state":"Georgia","postal_code":"31132"}},
{"student_id":5,"student_first_name":"Lilian","student_last_name":"Warret","student_email":"lwarret4@nsw.gov.au","student_gender":"Male","student_phone_numbers":["5031246553","6151432197","2152754201"],"student_address":{"street":"82540 Summer Ridge Point","city":"Sioux Falls","state":"South Dakota","postal_code":"57193"}},
{"student_id":6,"student_first_name":"Tate","student_last_name":"Swyne","student_email":"tswyne5@hud.gov","student_gender":"Agender","student_phone_numbers":["2021437429","8507115330","3047568052","7818031186","6072847440"],"student_address":{"street":"23 Sommers Parkway","city":"El Paso","state":"Texas","postal_code":"88569"}},
{"student_id":7,"student_first_name":"Ichabod","student_last_name":"Moring","student_email":"imoring6@un.org","student_gender":"Female","student_phone_numbers":["7147001301","9895085931"],"student_address":{"street":"584 Reindahl Way","city":"Denver","state":"Colorado","postal_code":"80228"}},
{"student_id":8,"student_first_name":"Ariel","student_last_name":"Howler","student_email":"ahowler7@tinypic.com","student_gender":"Agender","student_phone_numbers":null,"student_address":{"street":null,"city":null,"state":null,"postal_code":null}},
{"student_id":9,"student_first_name":"Octavia","student_last_name":"Stenner","student_email":"ostenner8@networksolutions.com","student_gender":"Bigender","student_phone_numbers":null,"student_address":{"street":null,"city":null,"state":null,"postal_code":null}},
{"student_id":10,"student_first_name":"Ronda","student_last_name":"Stean","student_email":"rstean9@xrea.com","student_gender":"Genderfluid","student_phone_numbers":null,"student_address":{"street":null,"city":null,"state":null,"postal_code":null}}]}
"""

In [ ]:
students2 = json.loads(students2_str)
students2_df = spark.createDataFrame(Row(**x) for x in students2['students'])

In [ ]:
mergecondition = student_delta.alias("t").merge(source = students2_df.alias("s"), condition = "s.student_id = t.student_id")

In [ ]:
type(mergecondition)// delta

In [ ]:
mergecondition = student_delta.alias("t").merge(source = students2_df.alias("s"), condition = "s.student_id = t.student_id")\
.whenMatchedUpdateAll()\
.whenNotMatchedInsertedAll()\// insert data from student2_df if data is not there but structure is same
.execute()
# if the structure is not same in target and other table then we have to specify in both update and inserted 

In [2]:
#if we want to update some of the firlds for the existing dATA 

mergecondition = student_delta.alias("t").merge(source = students2_df.alias("s"), condition = "s.student_id = t.student_id")\
.whenMatchedUpdateAll(  set = {
    'student_first_name ': 's.student_first_name'

})\
.whenNotMatchedInsertedAll()\// insert data from student2_df if data is not there but structure is same
.execute()

SyntaxError: unexpected character after line continuation character (99126591.py, line 7)